In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Function for Normalization
def normalize(series):
    min_val, max_val = series.min(), series.max()
    return (series - min_val) / (max_val - min_val) if max_val > min_val else 0

In [ ]:
#global variables
current_date = datetime.now().strftime("%Y%m%d")
COUNTRY_SCORE_WEIGHTS = {'Exports': 0.4, 'Imports': 0.3, 'Gross_Deals': 0.3}
CATEGORY_SCORE_WEIGHTS = {'Exports': 0.4, 'Imports': 0.3, 'Gross_Deals': 0.3}


# Load the dataset
df = pd.read_csv("final_data/20241122one_row_one_country_data.csv")

In [ ]:
# Compute country_metrics
exports = df[df['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
imports = df[df['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
gross_deals = exports.abs() + imports

# Normalize country_metrics
country_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
country_metrics['Normalized_Exports'] = normalize(country_metrics['Exports'])
country_metrics['Normalized_Imports'] = normalize(country_metrics['Imports'])
country_metrics['Normalized_Gross'] = normalize(country_metrics['Gross_Deals'])

# Compute composite score

country_metrics['Country_Score'] = round(
    COUNTRY_SCORE_WEIGHTS['Exports'] * country_metrics['Normalized_Exports'] +
    COUNTRY_SCORE_WEIGHTS['Imports'] * country_metrics['Normalized_Imports'] +
    COUNTRY_SCORE_WEIGHTS['Gross_Deals'] * country_metrics['Normalized_Gross']
    , 5
) * 100

# Sort by score
country_metrics = country_metrics.sort_values(by='Country_Score', ascending=False)
display(country_metrics)


In [ ]:
# Initialize an empty dictionary to store category-wise scores
category_scores = {}

# Get the unique categories
categories = df['Category'].unique()

for category in categories:
    # Filter data for the current category
    category_data = df[df['Category'] == category]
    
    # Compute Exports, Imports, and Gross Deals for the current category
    exports = category_data[category_data['Country Role'] == 'Supplier'].groupby('Country')['SIPRI TIV for total order'].sum()
    imports = category_data[category_data['Country Role'] == 'Recipient'].groupby('Country')['SIPRI TIV for total order'].sum().abs()
    gross_deals = exports.abs() + imports
    
    # Normalize category_metrics
    category_metrics = pd.DataFrame({'Exports': exports, 'Imports': imports, 'Gross_Deals': gross_deals}).fillna(0)
    category_metrics['Normalized_Exports'] = normalize(category_metrics['Exports'])
    category_metrics['Normalized_Imports'] = normalize(category_metrics['Imports'])
    category_metrics['Normalized_Gross'] = normalize(category_metrics['Gross_Deals'])
    
    # Define weights and calculate composite score
    category_metrics['Country_Score'] = round(
        CATEGORY_SCORE_WEIGHTS['Exports'] * category_metrics['Normalized_Exports'] +
        CATEGORY_SCORE_WEIGHTS['Imports'] * category_metrics['Normalized_Imports'] +
        CATEGORY_SCORE_WEIGHTS['Gross_Deals'] * category_metrics['Normalized_Gross']
        , 5
    ) * 100
    
    # Store the scores in the dictionary
    category_scores[category] = category_metrics[['Country_Score']]
    
# Combine all category scores into one DataFrame for analysis
final_category_scores = pd.concat(category_scores, axis=1)
final_category_scores.columns = [f"{category}_Score" for category in category_scores.keys()]

# Display the final scores
display(final_category_scores)

In [86]:
# Inner join based on 'Country'
result = pd.merge(country_metrics, final_category_scores, on='Country')
result_with_loop = pd.concat([result, final_category_scores['Aircraft_Score'].rename('ZZ_loop_score')], axis=1)
display(result_with_loop)

,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score,Aircraft_Score,Specialized Equipment_Score,Missiles/Rockets/Bombs_Score,Helicopters_Score,Unmanned Vehicles_Score,Ground Vehicles/Artillery_Score,Radar/Detection Systems_Score,Naval Vessels_Score,ZZ_loop_score
Country,,,,,,,,,,,,,,,,
United States,752270.38,42010.47,794280.85,1.000000,0.299696,1.000000,78.991,75.299,100.000,81.471,78.385,70.946,58.469,79.706,59.521,75.299
Soviet Union,373140.03,21974.87,395114.90,0.496019,0.156765,0.497450,39.467,30.641,1.421,21.605,23.605,21.845,75.672,21.230,47.469,30.641
India,1028.96,140176.72,141205.68,0.001368,1.000000,0.177778,35.388,34.443,17.726,33.551,18.269,30.000,36.725,39.884,40.934,34.443
Saudi Arabia,170.04,84180.94,84350.98,0.000226,0.600534,0.106198,21.211,21.109,4.623,34.481,12.800,12.466,19.034,17.351,8.707,21.109
United Kingdom,102732.99,42200.34,144933.33,0.136564,0.301051,0.182471,19.968,16.682,53.010,25.799,19.370,23.418,11.528,20.239,31.678,16.682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRC (Israel/Palestine)*,0.00,0.25,0.00,0.000000,0.000002,0.000000,0.000,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN
Provisional IRA (UK)*,0.00,1.20,0.00,0.000000,0.000009,0.000000,0.000,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN
RPF (Rwanda)*,0.00,0.35,0.00,0.000000,0.000002,0.000000,0.000,NaN,NaN,0.001,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
#Exporting the result to a csv file
result_with_loop.to_csv('final_data/'+current_date+'_country_score_data.csv')

In [88]:
result_with_loop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261 entries, United States to MTA (Myanmar)*
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Exports                          261 non-null    float64
 1   Imports                          261 non-null    float64
 2   Gross_Deals                      261 non-null    float64
 3   Normalized_Exports               261 non-null    float64
 4   Normalized_Imports               261 non-null    float64
 5   Normalized_Gross                 261 non-null    float64
 6   Country_Score                    261 non-null    float64
 7   Aircraft_Score                   194 non-null    float64
 8   Specialized Equipment_Score      156 non-null    float64
 9   Missiles/Rockets/Bombs_Score     193 non-null    float64
 10  Helicopters_Score                183 non-null    float64
 11  Unmanned Vehicles_Score          140 non-null    float64
 12  Grou

In [89]:
result_with_loop.describe()

,Exports,Imports,Gross_Deals,Normalized_Exports,Normalized_Imports,Normalized_Gross,Country_Score,Aircraft_Score,Specialized Equipment_Score,Missiles/Rockets/Bombs_Score,Helicopters_Score,Unmanned Vehicles_Score,Ground Vehicles/Artillery_Score,Radar/Detection Systems_Score,Naval Vessels_Score,ZZ_loop_score
count,261.000000,261.000000,261.000000,2.610000e+02,261.000000,261.000000,261.000000,194.00000,156.000000,193.000000,183.000000,140.000000,205.000000,137.000000,157.000000,194.00000
mean,7654.255632,7654.255632,15069.958123,1.017487e-02,0.054604,0.018973,2.614322,2.97816,3.021263,3.437166,3.194497,3.856071,3.635361,4.418839,5.657631,2.97816
std,54333.239315,16671.941776,59919.886720,7.222568e-02,0.118935,0.075439,7.083542,7.35454,10.129503,8.361781,7.515988,9.649840,8.668256,9.656414,11.309874,7.35454
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001000,0.007000,0.00000
25%,0.000000,30.990000,0.000000,0.000000e+00,0.000221,0.000000,0.008000,0.02325,0.026000,0.006000,0.121000,0.095000,0.038000,0.129000,0.076000,0.02325
50%,0.640000,402.280000,14.260000,8.507579e-07,0.002870,0.000018,0.091000,0.23700,0.153000,0.219000,0.636000,0.535000,0.404000,0.867000,0.673000,0.23700
75%,103.000000,7164.060000,7069.180000,1.369189e-04,0.051107,0.008900,1.805000,2.56850,1.420250,2.344000,3.026500,1.780250,2.958000,3.961000,6.230000,2.56850
max,752270.380000,140176.720000,794280.850000,1.000000e+00,1.000000,1.000000,78.991000,75.29900,100.000000,81.471000,78.385000,70.946000,75.672000,79.706000,73.271000,75.29900


In [90]:
# Melt the result DataFrame
melted_result = result_with_loop.reset_index().melt(
    id_vars=['Country'],  # Columns to keep as identifiers
    var_name='Metric Name',  # Name for the new column with metric names
    value_name='Metric Value'  # Name for the new column with metric values
).sort_values(by=['Country', 'Metric Name'])

# Display the melted DataFrame
display(melted_result)
melted_result.to_csv('final_data/'+current_date+'_country_score_data_melted.csv')

,Country,Metric Name,Metric Value
2062,ANC (South Africa)*,Aircraft_Score,NaN
1801,ANC (South Africa)*,Country_Score,0.000
235,ANC (South Africa)*,Exports,0.000
757,ANC (South Africa)*,Gross_Deals,0.000
3367,ANC (South Africa)*,Ground Vehicles/Artillery_Score,NaN
...,...,...,...
1181,unknown supplier(s),Normalized_Imports,0.000
3530,unknown supplier(s),Radar/Detection Systems_Score,0.002
2225,unknown supplier(s),Specialized Equipment_Score,0.266
3008,unknown supplier(s),Unmanned Vehicles_Score,0.149
